In [1]:
import pandas
import glob
import sqlalchemy
import numpy
print(pandas.__version__)

#from folium.plugins import MarkerCluster # for clustering the markers
import plotly.express as px
import plotly.graph_objects as go

# 
pandas.set_option('max_columns', None)

1.3.4


# Laden der einzelnen Datensätze unterschiedlicher .csv Daten

Als erste werden die Großhandelspreise eingeladen. Hierbei ist das famose Punkt-Komma Problem von deutschen tabellarischen Daten zu beachten!

In [2]:
# Path to csv files
CSV_PATH = r"/home/werleinm/Dokumente/Test-code/cfd/smard_data"

 # use your path
all_files = glob.glob(CSV_PATH  + "/Gro*.csv")

li = []

for filename in all_files:
    df = pandas.read_csv(filename, delimiter=";", na_values="-",decimal=",", parse_dates={"date":["Datum","Uhrzeit"]}, dayfirst=True)
    li.append(df)

df = pandas.concat(li, axis=0, ignore_index=True)

#df = pandas.read_csv(CSV_DATA_GRO_HANDEL,delimiter=";",decimal=",")
#df["Datum"] = pandas.to_datetime(df['Datum'] + df['Uhrzeit'], format='%d.%m.%Y%H:%M')
#df = df.drop(["Uhrzeit"], axis=1)

df = df.set_index("date")

for column in df.columns:
    if df[column].dtype == "object":
        df[column] = df[column].str.replace(".","")
        df[column] = df[column].str.replace(",",".")
        df[column] = df[column].astype("float")

grohandel = df[~df.index.duplicated(keep='first')]# #.resample("4h").sum() #.head(n=10)
grohandel = grohandel.rename(columns={"Deutschland/Luxemburg[€/MWh]":"Gro_handelspreis [€/MWh]"})
grohandel = grohandel["Gro_handelspreis [€/MWh]"]
grohandel.head()

date
2018-10-01 00:00:00    59.53
2018-10-01 01:00:00    56.10
2018-10-01 02:00:00    51.41
2018-10-01 03:00:00    47.38
2018-10-01 04:00:00    47.59
Name: Gro_handelspreis [€/MWh], dtype: float64

Als nächstes werden die Realisierten Erzeugerdaten hinzugefügt. 

In [3]:
 # use your path
all_files = glob.glob(CSV_PATH  + "/Real*.csv")

li = []

for filename in all_files:
    df = pandas.read_csv(filename, delimiter=";", na_values="-", decimal=",", parse_dates={"date":["Datum","Uhrzeit"]}, dayfirst=True)
    li.append(df)

df = pandas.concat(li, axis=0, ignore_index=True)

df = df.set_index("date")

for column in df.columns:
    if df[column].dtype == "object":
        df[column] = df[column].str.replace(".","")
        df[column] = df[column].str.replace(",",".")
        df[column] = df[column].astype("float")

Erzeugung = df[~df.index.duplicated(keep='first')]# #.resample("4h").sum() #.head(n=10)
Erzeugung = Erzeugung.resample("1h").mean()
Erzeugung.head()

/tmp/ipykernel_10111/2621575222.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column] = df[column].str.replace(".","")


,Biomasse[MWh],Wasserkraft[MWh],Wind Offshore[MWh],Wind Onshore[MWh],Photovoltaik[MWh],Sonstige Erneuerbare[MWh],Kernenergie[MWh],Braunkohle[MWh],Steinkohle[MWh],Erdgas[MWh],Pumpspeicher[MWh],Sonstige Konventionelle[MWh]
date,,,,,,,,,,,,
2018-10-01 00:00:00,1125.5625,319.7500,406.50,1060.50,0.0,36.0,1973.5625,3886.50,1986.5,845.75,59.25,405.50
2018-10-01 01:00:00,1118.3125,310.0625,521.25,989.75,0.0,36.0,1976.7500,3901.50,1940.5,812.25,49.00,401.75
2018-10-01 02:00:00,1119.7500,306.6250,536.25,969.25,0.0,36.0,1981.7500,3921.75,1889.5,832.25,49.00,404.25
2018-10-01 03:00:00,1117.1875,309.2500,550.50,1035.50,0.0,36.0,1959.2500,3938.25,1981.0,793.00,49.00,405.00
2018-10-01 04:00:00,1120.0625,308.8750,640.50,1145.75,0.0,36.0,1965.1875,3954.00,2055.5,840.75,49.00,406.50


Anschließend kombinieren wir beide Datensätze zu einem Dataframe.

In [4]:
dfz = pandas.concat([grohandel, Erzeugung["Photovoltaik[MWh]"]], axis=1)
dfz.head()
#dfz["neueSpalte"] = dfz["Photovoltaik[MWh]"] * dfz["DA_Sp"]#.plot()

,Gro_handelspreis [€/MWh],Photovoltaik[MWh]
date,,
2018-10-01 00:00:00,59.53,0.0
2018-10-01 01:00:00,56.10,0.0
2018-10-01 02:00:00,51.41,0.0
2018-10-01 03:00:00,47.38,0.0
2018-10-01 04:00:00,47.59,0.0


Nun definieren wir und ein dictionary mit von uns benötigten Marktwerten. Hier habe ich die Daten händisch zusammen kopiert, da das schneller ging als sie aus vielen unterschiedlichen Quellen mit einem Skript zusammen zu führen.

Aus dem `dict` bauen ich als erstes danach ein Pataframe mit den tatsächlichen Jahreswerten für PV. Pandas kann super aus übergebenen `dict`s ein Dataframe bauen, da DataFrames selber im endeffekt auch nichts anderes sind. Weitere Beschreibung im Code:

In [5]:
marktwerte = {
    "Jahresmarktwerte":{
        "Einheit":"ct/kWh",
        "PV":{
            "Jahr":[2020,2021],
            "Jahresmarktwert":[2.458,7.552]
        },
        "Wind":{
            "Jahr":[],
            "Jahresmarktwert":[]
        }
    },
    "tatsächlicher Jahresmarktwert":{
        "Einheit":"ct/kWh",
        "PV":{
            "Jahr":[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
            "Jahresmarktwert":[4.495,3.936,3.333,3.171,2.956,3.474,4.515,3.776,2.897,9.562, 0],
            "anzulegender Wert":[5,5,5,5,5,5,5,5,5,5, 0]
        },
        "Wind":{
            "Jahr":[],
            "Jahresmarktwert":[]
        }

    }
}

PV_marktwerte = pandas.DataFrame(marktwerte["tatsächlicher Jahresmarktwert"]["PV"])
PV_marktwerte

,Jahr,Jahresmarktwert,anzulegender Wert
0,2012,4.495,5
1,2013,3.936,5
2,2014,3.333,5
3,2015,3.171,5
4,2016,2.956,5
5,2017,3.474,5
6,2018,4.515,5
7,2019,3.776,5
8,2020,2.897,5
9,2021,9.562,5


Jetzt ist jeder Eintrag aus dem `dict` eine Spalte in dem neuen DataFrame. 
Als nächstes wollen wir das Datum als `datetime` Datentyp umwandeln und es anschließend als Index setzen, um es später mit unseren anderen Daten zu verschneiden:

In [6]:
PV_marktwerte["Jahr"] = pandas.to_datetime(PV_marktwerte["Jahr"], format="%Y")
PV_marktwerte = PV_marktwerte.rename(columns={"Jahr":"date"})
PV_marktwerte = PV_marktwerte.set_index("date")
PV_marktwerte

,Jahresmarktwert,anzulegender Wert
date,,
2012-01-01,4.495,5
2013-01-01,3.936,5
2014-01-01,3.333,5
2015-01-01,3.171,5
2016-01-01,2.956,5
2017-01-01,3.474,5
2018-01-01,4.515,5
2019-01-01,3.776,5
2020-01-01,2.897,5


Jetzt noch schnell ein Präfix angehangen, damit wir die Spalten später leichter identifizieren können und von ct/kWh in €/MWh umgerechnet:

In [7]:
PV_marktwerte = PV_marktwerte.add_prefix("PV_")
PV_marktwerte = PV_marktwerte.add_suffix(" [€/MWh]")
PV_marktwerte = PV_marktwerte / 100 * 1000
PV_marktwerte

,PV_Jahresmarktwert [€/MWh],PV_anzulegender Wert [€/MWh]
date,,
2012-01-01,44.95,50.0
2013-01-01,39.36,50.0
2014-01-01,33.33,50.0
2015-01-01,31.71,50.0
2016-01-01,29.56,50.0
2017-01-01,34.74,50.0
2018-01-01,45.15,50.0
2019-01-01,37.76,50.0
2020-01-01,28.97,50.0


Da die restlichen Daten jedoch eine höhere zeitliche Auflösung haben, müssen wir diese Daten auch noch final *upscalen*. Das bedeutet nichts anderes, als das wir sie wie zuvor die `df.resample()` funktion anwenden. Hier jedoch hin zu einer höheren zeitlichen Auflösung `"1Y"`  zu `"1h"`. Da wir hier nicht mit `.sum()` oder `.mean()` die entstandenen Lücken füllen können, nutzen wir eine sogenannte *forward fill* funktion. Diese hießt in Pandas `.ffill()` und fügt in Leerstellen den letzten Wert ein, bis ein neuer kommt.
Dann nehmen wir mit `[:-1]` noch den letzten Wert aus der Zeitreihe. Diesen für 2022 hatte ich nur eingefügt, damit im `.ffill()` auch das ganze Jahr 2021 aufgefüllt werden konnte. Ohne den einen Wert von 2022, wäre nach dem upscalen mit `.resample("1h").ffill()` nach dem `2021-01-01` schluss gewesen, da nicht zu einem nächst höheren Wert aufzufüllen gewesen wäre.

In [8]:
PV_marktwerte = PV_marktwerte.resample("1h").ffill()[:-1]
PV_marktwerte

,PV_Jahresmarktwert [€/MWh],PV_anzulegender Wert [€/MWh]
date,,
2012-01-01 00:00:00,44.95,50.0
2012-01-01 01:00:00,44.95,50.0
2012-01-01 02:00:00,44.95,50.0
2012-01-01 03:00:00,44.95,50.0
2012-01-01 04:00:00,44.95,50.0
...,...,...
2021-12-31 19:00:00,95.62,50.0
2021-12-31 20:00:00,95.62,50.0
2021-12-31 21:00:00,95.62,50.0


Nun füllen wir alle Daten zusammen und führen die Berechungen wie in deiner Excel-Tabelle durch. Das Verbinden machen wir hier zusätzlich über die Option *`join="inner"`*. Das bedeutet, dass Nur die Zeilen genommen werden, die laut dem Index in beiden DataFrames vorhanden sind. Hier Also Daten von 2018 - 2021. 

Anders als bei Excel, verwende ich hier für jahresspezifische Werte ganze Spalten anstatt einzelne Variablen. Das vereinfacht das Rechnen mit unterschiedlichen Spalten enorm, da du dann du in Pandas einfach gleich großen Spalten mit Basisopperationen wie `*, /, +, -, <, &, ...` verschneiden kannst.

Etwas anders ist nur wieder das Einbringen von Jahressummen. Hier erstelle ich die Jahressummen wieder über `df.resample("1Y).sum()`. Das Ergebniss ist jedoch ein DataFrame mit einem Wert pro Jahr. Hier müssen wir wieder die alte zeitliche Auflösung herstellen. Da wir aber nicht wieder ein Wert für 2022 haben, können wir nicht mit `.resample("1h").ffill()` arbeiten. Als Lösung Loope ich über jedes Jahr in dem zeitlich hoch aufgelösten `df` und fülle die Spalte `"Jahreserzeugung"` mit dem für der entsprechend gennerierten Jahressumme.

The `numpy.where()` function will in this case be our equivalent to excels `if` for a entire DataFrame.

In [9]:
dfz = pandas.concat([PV_marktwerte, dfz], axis=1, join="inner")#.loc["2021",:]
dfz["Auszahlung MP"] = numpy.where((dfz["PV_anzulegender Wert [€/MWh]"] > dfz["PV_anzulegender Wert [€/MWh]"]) & (dfz["PV_anzulegender Wert [€/MWh]"] > 0), dfz["PV_anzulegender Wert [€/MWh]"] - dfz["PV_Jahresmarktwert [€/MWh]"], 0)
dfz["Erlöse [€/MWh]"] = numpy.where(dfz["Photovoltaik[MWh]"] == 0, 0, dfz["Gro_handelspreis [€/MWh]"] + dfz["Auszahlung MP"])

dfz["Jahreserzeugung"] = None
jahreswerte = dfz["Photovoltaik[MWh]"].resample("Y").sum()

for year in jahreswerte.index.year:
    dfz.loc[str(year),"Jahreserzeugung"] = jahreswerte.loc[str(year)].values[0]

dfz["Erlös gesammt"] = dfz["Photovoltaik[MWh]"] * dfz["Erlöse [€/MWh]"]
dfz["Erlös nominal"] = dfz["Erlös gesammt"] / dfz["Jahreserzeugung"]

dfz["CFD-MP"] = dfz["PV_anzulegender Wert [€/MWh]"] - dfz["PV_Jahresmarktwert [€/MWh]"]

dfz.head(n=10)

,PV_Jahresmarktwert [€/MWh],PV_anzulegender Wert [€/MWh],Gro_handelspreis [€/MWh],Photovoltaik[MWh],Auszahlung MP,Erlöse [€/MWh],Jahreserzeugung,Erlös gesammt,Erlös nominal,CFD-MP
date,,,,,,,,,,
2018-10-01 00:00:00,45.15,50.0,59.53,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 01:00:00,45.15,50.0,56.10,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 02:00:00,45.15,50.0,51.41,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 03:00:00,45.15,50.0,47.38,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 04:00:00,45.15,50.0,47.59,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 05:00:00,45.15,50.0,51.61,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 06:00:00,45.15,50.0,69.13,0.00,0.0,0.00,1111155.75,0.00,0.0,4.85
2018-10-01 07:00:00,45.15,50.0,77.32,52.25,0.0,77.32,1111155.75,4039.97,0.003636,4.85
2018-10-01 08:00:00,45.15,50.0,84.97,502.00,0.0,84.97,1111155.75,42654.94,0.038388,4.85


Next we want to analyze a condition on *n* prior time steps in out Dataframe. For example:

Was the generated Power by PV during the last *4 hours* was greater than *some value*?

Therefore we are going to use the `.rolling()` function of a `pandas.Dataframe`. This function will apply a moving window to our passed `Dataframe` (2D) or Series (1D). The size of the window is given by the `window=` argument and can either be an `int` representing a number of raws or columns, or a `str` in a valid timeframe format (`4h` for 4 hours, `4d` for 4 days, ...). 
Next you can chose if you want the window to apply for all prior values (`center = False`), or all surrounding values (`center = True`). Finally we can chose whether we want to include the first or last value of the window with the `closed=` argument. 

If `right`, the first point in the window is excluded from calculations.

If `left`, the last point in the window is excluded from calculations.

If `both`, the no points in the window are excluded from calculations.

If `neither`, the first and last points in the window are excluded from calculations.

After the `.rolling()` function we lastly need to specify a aggregation function to use on the vales inside the window. Basic functions would be `.sum()`, `.mean()`, `.median()`, `.count()`.

In [49]:
# In order to let you experiment, I put both the old and new column side by side
timeframe = "4h"

test = dfz["Photovoltaik[MWh]"].rolling(window=timeframe, center=False, closed="right").sum()
test.name = f"Photovoltaik[MWh]_prior_{timeframe}_sum"

pandas.concat([test,dfz["Photovoltaik[MWh]"]], axis=1).tail(n=20)

,Photovoltaik[MWh]_prior_4h_sum,Photovoltaik[MWh]
date,,
2021-12-31 04:00:00,0.00,0.00
2021-12-31 05:00:00,0.00,0.00
2021-12-31 06:00:00,0.00,0.00
2021-12-31 07:00:00,0.00,0.00
2021-12-31 08:00:00,81.00,81.00
2021-12-31 09:00:00,807.75,726.75
2021-12-31 10:00:00,2459.25,1651.50
2021-12-31 11:00:00,4888.50,2429.25
2021-12-31 12:00:00,7309.25,2501.75


Next we are going to use the the generated column to filter hours, were the defined prior timeframe exceeds a specific value.

In [58]:
df_filterd = pandas.concat([test,dfz], axis=1)
df_filterd["Erlös limitiert"] = numpy.where(df_filterd["Photovoltaik[MWh]_prior_4h_sum"] > 8000, 0, df_filterd["Erlös gesammt"])

df_filterd[["Photovoltaik[MWh]","Photovoltaik[MWh]_prior_4h_sum","Erlös gesammt","Erlös limitiert"]].head(n=20)

,Photovoltaik[MWh],Photovoltaik[MWh]_prior_4h_sum,Erlös gesammt,Erlös limitiert
date,,,,
2018-10-01 00:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 01:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 02:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 03:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 04:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 05:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 06:00:00,0.00,0.00,0.0000,0.0000
2018-10-01 07:00:00,52.25,52.25,4039.9700,4039.9700
2018-10-01 08:00:00,502.00,554.25,42654.9400,42654.9400


Finally we are will retreve some yearly stats.

First, the number of hours where the threshhold of 8000 MWh/4h prior was exceeded. Therefore, we replace `0` values with `numpy.nan`, since these values will not be taken in account when counting the raws.

In [63]:
df_filterd["Erlös limitiert"].replace({0:numpy.nan}).resample("1Y").count()

date
2018-12-31     788
2019-12-31    2828
2020-12-31    2790
2021-12-31    2719
Freq: A-DEC, Name: Erlös limitiert, dtype: int64

Next we calculate the difference in revenue / 1'000'000 per year between not limited and limited for 8000 MWh/4h prior.

In [62]:
(df_filterd["Erlös gesammt"].resample("1Y").sum() - df_filterd["Erlös limitiert"].resample("1Y").sum()) / 1000000

date
2018-12-31     28.279520
2019-12-31    262.929100
2020-12-31    196.868338
2021-12-31    636.069194
Freq: A-DEC, dtype: float64